In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import fisher_exact

In [ ]:
save_files = False

In [ ]:
results_dir = "../../data/results_files/"
pd.set_option("display.max_rows", 5)

In [ ]:
res = pd.read_csv(results_dir + "results.csv", index_col=0)
emmeans = pd.read_csv(results_dir + "emmeans.csv", index_col=0)

# See which fit were singular and eliminate them

Fits for which some of the random effects were not considered

In [ ]:
len(res[res["israndom"] == "fixed"])  # Number of fixed effect models

In [ ]:
len(res[res["israndom"] == "O"])

In [ ]:
len(res[res["israndom"] == "C"])

In [ ]:
len(res[res["israndom"] == "OC"])

There are 180 reactions over 7605 for which no random effects were significant, 126 for which the Chip effect was not significant, and 2002 for which the origin random effect was not significant.

# Convergence and quality of the fit

In [ ]:
errors = res[
    res["fit msg"]
    != "E fixed-effect model matrix is rank deficient so dropping 4 columns / coefficients"  # This is an expected message
]
np.unique(errors["fit msg"])

No error to report

In [ ]:
plt.hist(res["r2c"], alpha=0.5, label="Conditional R2")
plt.hist(res["r2m"], alpha=0.5, label="Marginal R2")
plt.title("R2")
plt.legend()

# Integrate the emmeans estimates and pvalue corrections

In [ ]:
full_res = res.merge(emmeans, left_index=True, right_index=True)

In [ ]:
# Get the vector of pvalues

pval_col = [
    column
    for column in list(full_res.columns)
    if (
        "pval" in column and "Model" not in column and "Intercept" not in column
    )  # Remove intercept from pvalue correction
]
pvalues_lists = [list(full_res[col]) for col in pval_col]
pvalues = [item for sublist in pvalues_lists for item in sublist]
from statsmodels.stats.multitest import fdrcorrection, multipletests

pvalues_BH = multipletests(pvalues, 0.05, method="holm")[1]

plt.hist(np.log10(pvalues_BH))
plt.title("Holm corrected log10 p-values for the emmeans estimates")

In [ ]:
[c for c in full_res.columns if "pval" in c]

In [ ]:
# Re create the pvalues columns

i = 0
BH_col = []
for col in full_res.columns:
    if (
        "pval" in col
        and "Model" not in col
        and "Intercept" not in col  # Remove intercept from pvalue correction
    ):  # Keep only the emmeans, loose the Model estimates directly output by the MM
        new_name = col + " BH"
        beg = i * len(full_res)
        end = (i + 1) * len(full_res)
        l = pvalues_BH[beg:end]
        full_res[new_name] = l
        i += 1

full_res

In [ ]:
full_res.columns = [
    "Contrast " + c.replace("vs", "-") if "vs" in c else c for c in full_res.columns
]

# Match to the reactions names and subsystems

In [ ]:
subsystems = pd.read_csv(
    "../../data/processed_files/subsystem_duplicated.csv",
    index_col=0,
)
rxn = pd.read_csv("../../data/processed_files/rxn_names.csv", index_col=0)
rxn.columns = ["name"]

In [ ]:
full_res["Reaction"] = rxn["name"]
subsystem_col = []

for r in full_res.index:
    subsys = subsystems[subsystems["rxn"] == r]["level3"]
    subsys_list = list(subsys)
    if len(subsys_list) == 0:
        subsys_list = ["Unknown"]
    subsystem_col.append(subsys_list)
subsystem_col = pd.Series(subsystem_col, index=full_res.index)
full_res["Subsystem"] = subsystem_col

# Plot the results


This is generates the data for Figure 3.d and Extended Figures 6f, 6g and 6h.

In [ ]:
full_res.columns

In [ ]:
aplha = 0.05
boxplot_matrix = []
for reaction_name in full_res.index:
    for contrast in [
        "Contrast Wild - SPF",
        "Contrast Wild - Human",
        "Contrast SPF - Human",
        "MouseGenderMale",
        "HumanGenderMale",
        "MouseAdult_PupsPup",
    ]:
        pval = full_res.loc[reaction_name, contrast + " pval BH"]
        for subsys in full_res.loc[reaction_name, "Subsystem"]:
            boxplot_matrix.append(
                [
                    subsys,
                    contrast,
                    pval,
                    full_res.loc[reaction_name, contrast + " est"],
                ]
            )


boxplot_df = pd.DataFrame(
    boxplot_matrix, columns=["Subsystem", "Effect", "Corrected pvalue", "Estimate"]
)

TP: considered subsystem and significant
FP: significant, other subsystem
FN: not significant, considered subsystem
TN: not significant, other subsystem

In [ ]:
# Create one df per contrast
correct = True

contrast_df_dic = {}
for contrast in [
    "Contrast Wild - SPF",
    "Contrast Wild - Human",
    "Contrast SPF - Human",
    "MouseGenderMale",
    "HumanGenderMale",
    "MouseAdult_PupsPup",
]:
    contrast_df = boxplot_df[boxplot_df["Effect"] == contrast]
    contrast_df["ones"] = pd.Series([1] * len(contrast_df), index=contrast_df.index)
    subsys_count = contrast_df.groupby("Subsystem").count()["ones"]
    subsystems_all = np.unique(contrast_df["Subsystem"])
    subsystems_enriched = []
    for (
        s
    ) in (
        subsystems_all
    ):  # We consider the reactions with unknown subsystem for the Fisher test but we remove them from the final df for plotting
        df_sig = contrast_df[contrast_df["Corrected pvalue"] <= 0.05]
        df_nonsig = contrast_df[contrast_df["Corrected pvalue"] > 0.05]
        # for each subsystem, check if it is enriched
        contingency_mat = np.zeros((2, 2))
        contingency_mat[0, 0] = len(df_sig[df_sig["Subsystem"] == s])
        contingency_mat[0, 1] = len(df_sig) - contingency_mat[0, 0]
        contingency_mat[1, 0] = len(df_nonsig[df_nonsig["Subsystem"] == s])
        contingency_mat[1, 1] = len(df_nonsig) - contingency_mat[1, 0]
        res = fisher_exact(contingency_mat, alternative="greater")
        # No pvalue correction at this step
        # Made explicit in the manuscript methods.
        if res.pvalue <= 0.05:
            subsystems_enriched.append(s)
            print(res.pvalue)
    contrast_df = boxplot_df[boxplot_df["Effect"] == contrast]
    contrast_df = contrast_df[
        contrast_df["Subsystem"] != "Unknown"
    ]  # Remove the "Unknown" subsystem from plotting
    contrast_df = contrast_df[contrast_df["Subsystem"].isin(subsystems_enriched)]
    # Plot only the significant effects
    contrast_df = contrast_df[contrast_df["Corrected pvalue"] <= 0.05]
    contrast_df_dic[contrast] = contrast_df

In [ ]:
# Plots


def boxplot(
    df, contrast, figsize, ymax, effect_thresh=0.01, ymin=-1, swarmplot=True, title=None
):
    plt.figure(figsize=figsize)
    sns.boxplot(data=df, y="Subsystem", x="Estimate")
    if swarmplot:
        sns.swarmplot(data=df, y="Subsystem", x="Estimate", color="k", s=5)
    plt.vlines(
        [0], ymin=ymin, ymax=ymax, colors=["g"], linestyles=["dashed"], label="0"
    )
    plt.vlines(
        [-effect_thresh, effect_thresh],
        ymin=ymin,
        ymax=ymax,
        colors=["orange"],
        linestyles=["dashed"],
        label=f"+/- {effect_thresh}",
    )

    if title is not None:
        plt.title(title)
    else:
        plt.title(contrast)
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

In [ ]:
contrasts = [
    "Contrast Wild - SPF",
    "Contrast Wild - Human",
    "Contrast SPF - Human",
    "MouseGenderMale",
    "HumanGenderMale",
    "MouseAdult_PupsPup",
]

In [ ]:
contrast_df_dic[contrasts[-1]]

In [ ]:
contrast_df_dic[contrasts[5]]["Subsystem"].unique()

In [ ]:
boxplot(
    contrast_df_dic[contrasts[0]],
    contrast=contrasts[0],
    figsize=(5, 3),
    ymax=2,
    swarmplot=True,
)

In [ ]:
boxplot(contrast_df_dic[contrasts[1]], contrast=contrasts[1], figsize=(5, 3), ymax=3)

In [ ]:
boxplot(contrast_df_dic[contrasts[2]], contrast=contrasts[2], figsize=(5, 5), ymax=6)

In [ ]:
boxplot(
    contrast_df_dic[contrasts[3]],
    contrast=contrasts[3],
    figsize=(5, 8),
    ymax=17,
    title="Contrast Male - Female (for mice)",
)

In [ ]:
boxplot(
    contrast_df_dic[contrasts[4]],
    contrast=contrasts[4],
    figsize=(5, 2),
    ymax=1,
    title="Contrast Male - Female (for humans)",
)

In [ ]:
boxplot(
    contrast_df_dic[contrasts[5]],
    contrast=contrasts[5],
    figsize=(5, 8),
    ymax=14,
    title="Contrast Pup - Adults (for mice)",
)

# Additional test

Is the glyoxylate subsystem more enriched in significant reactions than the other subsystems?

In [ ]:
for contrast in [
    "Contrast Wild - SPF",
    "Contrast Wild - Human",
    "Contrast SPF - Human",
    "MouseGenderMale",
    "HumanGenderMale",
    "MouseAdult_PupsPup",
]:
    contrast_df = boxplot_df[boxplot_df["Effect"] == contrast]

    glyoxy = contrast_df[
        contrast_df["Subsystem"] == "Glyoxylate and dicarboxylate metabolism"
    ]
    other = contrast_df[
        contrast_df["Subsystem"] != "Glyoxylate and dicarboxylate metabolism"
    ]
    glyoxy_sig = glyoxy[glyoxy["Corrected pvalue"] <= 0.05]
    glyoxy_nonsig = glyoxy[glyoxy["Corrected pvalue"] > 0.05]
    other_sig = other[other["Corrected pvalue"] <= 0.05]
    other_nonsig = other[other["Corrected pvalue"] > 0.05]

    contingency_mat = np.zeros((2, 2))
    contingency_mat[0, 0] = len(glyoxy_sig)
    contingency_mat[0, 1] = len(other_sig)
    contingency_mat[1, 0] = len(glyoxy_nonsig)
    contingency_mat[1, 1] = len(other_nonsig)
    res = fisher_exact(contingency_mat, alternative="greater")
    print(contrast, res)

# Save the contrasts dataframes: This is the data for Figure 3d and Supplementary Figures 6f, 6g and 6h

In [ ]:
if save_files == True:
    for contrast in contrasts:
        df = contrast_df_dic[contrast]
        df.to_csv(f"../../data/manuscript/Main_figure_results_{contrast}.csv")